<h3>Add Basic Imports</>

In [37]:
import sys
sys.path.insert(0, '../')
import numpy as np
from qhdopt import QHD

<h3> Helper function to create Sympy function from QP file </>

In [2]:
def get_function_from_qp_file(dimension):
    with open(f"{dimension}d_instance.npy", 'rb') as f:
        Q = np.load(f)
        b = np.load(f)
    return QHD.quad_to_gen(Q, b)

<h3> Defining list of functions</>

In [3]:
from sympy import symbols, log, sin, cos, exp, expand, lambdify, N
from sympy.core.evalf import evalf

# Define the symbols
x, y, z = symbols('x y z')

functions = [get_function_from_qp_file(50),
            get_function_from_qp_file(60),
            get_function_from_qp_file(75),
            (-4*x**2 + 3*x*y - 2 * y ** 2 + 3 * x - y, [x, y]), # penalty = 1
             (-2*(x-1/3)**2 + y**2 -y*log(3*x + 1/2)/3 + 5*(x**2 - y**2 - x - 0.5)**2, [x, y]),
             (y**1.5 - (y-0.75)*exp(4*x),  [x, y]),
             (x**2 - 3*y**2 + 2*sin(1.5*np.pi*x) *cos(1.5*np.pi*y) + x - 3*y, [x, y]),
             ((2*y-1)**2*(z-0.4) - (2*x-1)*z + (2*x-1.5)**2*y, [x, y, z]),
             (2*exp(-x) * (2*z-1)**2 - 3*(2*y-0.7)**2 * exp(-z) + log(x + 1) * (y-0.8), [x, y, z]), # penalty = 1.2
             (x * sin(2*np.pi*y) - 0.5*(y-0.3)**2 * cos(3*np.pi*z) + sin(4*np.pi*x), [x, y, z]), # penalty = 1
             ]

<h3> List of parameters </>

In [38]:
resolution = 8
anneal_schedule = [[0,0],[20, 1]]
numruns = 1000
backend = "dwave"
tol = 1e-3
penalty_coefficient = 3e-2
api_key = "KEY_GOES_HERE"
embedding_scheme = "unary"

<h3> Helper function to calculate TTS </>

In [5]:
success_prop = 0.99
def TTS(t_f, p_s):
    p_s = min(p_s, success_prop)
    return t_f*(np.log(1 - success_prop) / (np.log(1 - p_s)))

In [16]:
def count_success_events(best, samples, f, tol):
    number_success = 0
    for sample in samples:
        if sample is not None and abs(f(sample).item() - best) < tol:
            number_success +=1
    return number_success

<h3> Function to run benchmark for a single problem </>

In [34]:
def run_test(func, syms, tol, penalty_ratio):
    data_vector = np.zeros(12)
    model = QHD.SymPy(func, syms)
    model.dwave_setup(resolution, numruns, api_key=api_key, embedding_scheme=embedding_scheme, anneal_schedule=anneal_schedule, penalty_coefficient=penalty_coefficient, penalty_ratio=penalty_ratio, post_processing_method="IPOPT")
    # Run QHD with post-processor = Ipopt
    current_best_qhd = model.optimize()
    qhd_ipopt_success_count = count_success_events(current_best_qhd, model.post_processed_samples, model.f_eval, tol)
    data_vector[0] = model.info["average_qpu_time"]
    data_vector[1] = model.info["post_processing_time"]
    data_vector[2] = qhd_ipopt_success_count/model.shots
    data_vector[3] = TTS(data_vector[0] + data_vector[1], data_vector[2])

    # Same QHD samples post-processed by TNC
    model.post_processing_method = "TNC"
    model.post_process()
    qhd_tnc_success_count = count_success_events(current_best_qhd, model.post_processed_samples, model.f_eval, tol)
    data_vector[4] = model.info["average_qpu_time"]
    data_vector[5] = model.info["post_processing_time"]
    data_vector[6] = qhd_tnc_success_count/model.shots
    data_vector[7] = TTS(data_vector[4] + data_vector[5], data_vector[6])

    # Run Ipopt with random init
    model.qhd_samples = np.random.rand(model.shots, model.dimension)
    model.post_processing_method = "IPOPT"
    _, current_best, post_processing_time = model.post_process()
    ipopt_success_count = count_success_events(current_best_qhd, model.post_processed_samples, model.f_eval, tol)
    data_vector[8] = post_processing_time
    data_vector[9] = ipopt_success_count/model.shots

    # Run TNC with random init
    model.post_processing_method = "TNC"
    model.num_occurrences = np.ones(model.shots, dtype=int)
    _, _, post_processing_time = model.post_process()
    tnc_success_count = count_success_events(current_best_qhd, model.post_processed_samples, model.f_eval, tol)

    data_vector[10] = post_processing_time
    data_vector[11] = tnc_success_count/model.shots

    return data_vector


<h3> Running benchmark on 10 problems</>

## Penalty Ratio = 1

In [20]:
penalty_ratio = 1
data = np.zeros((len(functions), 12))
for i in range(len(functions)):
    print(f"Test problem {i}:")
    function, free_vars = functions[i]
    data[i] = run_test(function, free_vars, tol, penalty_ratio)

Test problem 0:


KeyboardInterrupt: 

## Penalty Ratio = 1.2

In [39]:
penalty_ratio = 1.2
data = np.zeros((len(functions), 12))

In [35]:
for i in range(3, len(functions)):
    print(f"Test problem {i}:")
    function, free_vars = functions[i]
    data[i] = run_test(function, free_vars, tol, penalty_ratio)

Test problem 3:
Test problem 4:


KeyboardInterrupt: 